# Metrices to measure the grouping results from weighted PCA scores

The extreme divergent trends in the reference and degraded groups are not the only result we want to capture, it can cause issues of PCs driven by single or a few features.

Therefore, how to balance the overall grouping structure and the extreme trends is important. 
It needs to think of weight assignment carefully.

## Weights and composite scores of sites

Prepare the data

In [142]:
# magic commands to make sure external modules are reloaded every complete run
%load_ext autoreload
%autoreload 2

import pandas as pd
from zci.data_process.dataframe_ops import get_block
from sklearn.preprocessing import StandardScaler

# read the merged dataframe
master = pd.read_excel("../data/processed/complete_env_taxa_chemical.xlsx", 
                      sheet_name="all_data_merged", 
                      header=[0, 1, 2], 
                      index_col=0)

master.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


block     sample_info                           environmental            \
subblock          raw                                     raw             
var          Latitude Longitude Waterbody  Year       LOI (%) MPS (Phi)   
StationID                                                                 
A10          42.90455 -82.46760       SCR  2004      3.436193  1.734259   
A23          42.56658 -82.57595       LSC  2004      3.226637  1.969984   
A27          42.56007 -82.42132       LSC  2004      2.802642  1.319519   
A28          42.54577 -82.42073       LSC  2004      3.201399  1.398687   
A29          42.51440 -82.43462       LSC  2004      6.180718  1.065748   

block                                                                      \
subblock                                                                    
var       Measured Depth (m) Temperature (oC) Velocity  at bottom (m/sec)   
StationID                                                                   
A10                      1.8            19.17                         NaN   
A23                      3.0            19.10                         NaN   
A27                      1.0            18.40                         NaN   
A28                      0.5            18.90                         NaN   
A29                      0.5            19.30                         NaN   

block                             ...           taxa                          \
subblock                          ...            raw                           
var       Water DO Bottom (mg/L)  ... Hydropsychidae      Hydrozoa  Nematoda   
StationID                         ...                                          
A10                        10.21  ...   3.203427e-16  3.203427e-16  1.727234   
A23                        10.09  ...   3.203427e-16  3.203427e-16  2.427993   
A27                        10.30  ...   3.203427e-16  3.203427e-16  1.802219   
A28                        12.80  ...   3.203427e-16  3.203427e-16  2.771800   
A29                         9.70  ...   3.203427e-16  3.203427e-16  4.616441   

block                                                                \
subblock                                                              
var       Oligochaeta Other Trichoptera   Sphaeriidae   Turbellaria   
StationID                                                             
A10          6.576942      3.203427e-16  3.203427e-16  3.203427e-16   
A23          5.872905      3.203427e-16  1.022139e+00  5.997595e-01   
A27          6.402280      1.580169e+00  1.317615e+00  9.964067e-01   
A28          5.867874      2.049287e+00  2.863547e+00  3.203427e-16   
A29          5.181664      3.203427e-16  3.203427e-16  3.203427e-16   

block     2008_results                          
subblock   DR_clusters       corridor_clusters  
var         DR_cluster if_RF  corridor_cluster  
StationID                                       
A10                  0     0                 1  
A23                  0     0                 1  
A27                  0     0                 1  
A28                  0     0                 1  
A29                  0     0                 1  

[5 rows x 59 columns]

Standardize the stressor features: z-score

In [143]:
# take the stressor data block
stressor = get_block(master, block="chemical", subblock= "raw")

# transform the stressor features: standardization (z-score)
scaler = StandardScaler()
stressor_standardized = pd.DataFrame(scaler.fit_transform(stressor), 
                       columns=stressor.columns, 
                       index=stressor.index)

Set weights for groups of variables

In [144]:
# import the chemical weighting utilities
from zci.sediment_pollution_assessment.chemical_weights import (
    build_weights_for_columns, # function to build weight mapping for variables
    VARIABLE_TYPE_BY_NAME, # default category mapping for: variable name -> variable type
)

# define weights for different variable types
chem_cols = stressor.columns.tolist()

# specify custom weights for certain types (others will use defaults)
std_weight = 1

type_weights = {
    "Trace Metal (pollutant)":  1 * std_weight,
    "Hydrocarbon pollutant": 1 * std_weight,        # Highest priority: petroleum/chlorobenzenes
    "organochlorine pesticide": 1 * std_weight,     # Highest priority: POPs
    "Sum of all PCBs": 1 * std_weight,              # Highest priority: PCBs
    "Binding agent": 1 * std_weight,               # Medium priority: affects bioavailability
    "Earth element (nontoxic)": 1 * std_weight   # Lowest priority: background elements
}

# Build the final weight map for the variables for later use in weighted PCA
custom_weight_map = build_weights_for_columns(chem_cols,
                                              variable_type_by_name =VARIABLE_TYPE_BY_NAME,
                                              type_weights=type_weights,
                                              weights_by_name={"p,p'-DDD": 1 * std_weight,
                                                               "Cd": 3 * std_weight} # set a subtle weight for As specifically
                                             )

The pipeline to produce: X, weights -> composite scores

In [145]:
from zci.sediment_pollution_assessment.weighted_pca import WeightedPCA_Scores

weighted_PCA_grader = WeightedPCA_Scores(
    custom_weight_map,
    weight_threshold=2.0,
    group_thresholds=(0.2, 0.8)
)

# compute the weighted PCA scores
scores_with_labels =weighted_PCA_grader.fit_transform(stressor_standardized)


=== Selected Principal Components ===
PC       Explained Var   High-Weighted Variable Loadings
--------------------------------------------------------------------------------
PC1      0.3844          Cd: 0.686
PC2      0.1863          Cd: 0.477
--------------------------------------------------------------------------------
Total selected PCs: 2
High-weighted variables (1): Cd
Weights of selected PCs: PC1: 2.718, PC2: 2.718


## PERMANOVA from skbio library

In [146]:
import numpy as np
from skbio.stats.distance import permanova, DistanceMatrix
# compute the distance matrix
from scipy.spatial.distance import pdist, squareform

X = get_block(master, block="chemical", subblock= "raw").dropna(axis = 1)

# Filter out columns with high weights (>= weight_threshold)
# high_weight_cols = [col for col, weight in custom_weight_map.items() if weight >= weighted_PCA_grader.weight_threshold]
# X = X.drop(columns=high_weight_cols)
# print(high_weight_cols)
X = np.log1p(X + 1)  # log-transform to reduce skewness
groups = scores_with_labels['pollution_quality']
dist_mat = squareform(pdist(X, metric='euclidean'))

# Convert to skbio DistanceMatrix
dm = DistanceMatrix(dist_mat, ids=X.index)

# perform PERMANOVA
permanova_results = permanova(dm, groups, permutations=999)
print(permanova_results)

method name               PERMANOVA
test statistic name        pseudo-F
sample size                     104
number of groups                  3
test statistic             7.369423
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object


## Measure the representativeness of the selected PCs

In [147]:
from zci.sediment_pollution_assessment.ordination_metrices import evaluate_weighted_pca_representativeness
from sklearn.decomposition import PCA

representative_results = evaluate_weighted_pca_representativeness(weighted_PCA_grader)
print(f"Similarity set: {representative_results['similarity_set']}")
print(f"Representativeness: {representative_results['representativeness']}")

Similarity set: [np.float64(0.5619361122886893), np.float64(0.6752521184204505)]
Representativeness: 0.9582102395193982


## Measure the discrimination of the selected PCs

In [148]:
from zci.sediment_pollution_assessment.ordination_metrices import evaluate_weighted_pca_discrimination

discrimination_metrics_results = evaluate_weighted_pca_discrimination(weighted_PCA_grader)
discrimination_metrics_results["t_test_p_values_hw"]

[{'variable': 'Cd',
  'D_mean': np.float64(-0.360910622193962),
  't_statistic': np.float64(-2.468466467970999),
  'p_value': np.float64(0.008970298199460883),
  'significant_at_0.01': np.True_}]

In [149]:
from zci.sediment_pollution_assessment.ordination_metrices import groupby_aggregation
groupby_aggregation(X, scores_with_labels['pollution_quality'])

quality_label            reference     medium   degraded
var                                                     
%OC                mean   1.457942   1.348633   1.445039
                   std    0.359119   0.278268   0.268877
1234-TCB           mean   0.800837   0.791338   0.830602
                   std    0.124204   0.103841   0.115759
1245-TCB           mean   0.893477   0.959485   1.189989
                   std    0.273871   0.343117   0.519660
Al                 mean   8.811949   8.262777   8.307188
                   std    0.763618   0.551352   0.474576
As                 mean   1.585231   1.253955   1.242928
                   std    0.309820   0.329596   0.385767
Bi                 mean   1.735900   2.904479   2.863624
                   std    1.137863   0.690420   1.061441
Ca                 mean  10.064331  10.259946  10.387161
                   std    0.607641   0.457182   0.438166
Cd                 mean   0.758157   0.839991   1.010805
                   std    0.068051   0.068721   0.184279
Co                 mean   1.992613   1.725063   1.821743
                   std    0.311422   0.236783   0.234469
Cr                 mean   2.697902   2.344795   2.602761
                   std    0.538345   0.384423   0.792105
Cu                 mean   2.883126   2.438151   2.674403
                   std    0.727159   0.684738   1.060066
Fe                 mean   9.509100   8.995436   9.137991
                   std    0.567254   0.429052   0.431346
HCB                mean   1.303476   1.300499   2.176119
                   std    1.231729   0.616033   1.216137
Heptachlor Epoxide mean   0.706763   0.701471   0.794044
                   std    0.045854   0.029959   0.178613
Hg                 mean   0.755533   0.787972   1.000687
                   std    0.053275   0.106812   0.331610
K                  mean   7.032433   6.607857   6.754967
                   std    0.917509   0.653207   0.552711
Mg                 mean   9.296493   9.492067   9.514568
                   std    0.467676   0.441438   0.413699
Mn                 mean   5.293811   5.027411   5.099602
                   std    0.514336   0.389776   0.352067
Na                 mean   4.892231   4.771479   4.852447
                   std    0.353366   0.327683   0.354253
Ni                 mean   2.687860   2.346502   2.608744
                   std    0.540172   0.406125   0.573117
OCS                mean   0.932897   0.944887   1.179315
                   std    0.427887   0.366973   0.803304
Pb                 mean   2.002674   1.833699   2.366446
                   std    0.602630   0.553781   1.085839
QCB                mean   0.959624   0.942026   1.180144
                   std    0.512111   0.249624   0.454284
Sb                 mean   1.613673   2.796158   2.916987
                   std    1.122937   0.757286   1.064481
V                  mean   3.136764   2.786184   2.821743
                   std    0.464893   0.339021   0.312769
Zn                 mean   3.692799   3.527762   4.010523
                   std    0.608593   0.502415   0.721691
mirex              mean   0.731887   0.778416   0.823021
                   std    0.100113   0.121271   0.243664
p,p'-DDD           mean   0.978109   1.014283   1.103660
                   std    0.206452   0.322184   0.816508
p,p'-DDE           mean   0.844631   0.883541   0.917114
                   std    0.091965   0.214431   0.329593
total PCB          mean   1.517376   1.773384   2.663133
                   std    0.557977   0.717511   1.245497